# Gesture Recognition - Model 4
#### Conv3D Model with comprehensive preprocessing (using OpenCV library - Resize and Data augmetation with Affine transformation, Horizontal Flip, Cropping and Resizing,  Usage of Optimizer, ReduceLROnPlateau to reduce learning rate, and  experimenting with various sizes of batch sizes and Epochs)
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os
import cv2

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

Using TensorFlow backend.


In this block, you read the folder names for training and validation. We will set the initial `batch_size` here. Note that we will experimentally set the batch size in such a way that you are able to use the GPU in full capacity. Keep increasing the batch size until the machine throws an error.

In [3]:
train_doc = np.random.permutation(open('./Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('./Project_data/val.csv').readlines())
batch_size = 10 #experiment with the batch size

## Generator
This is one of the most important part of the code. The overall structure of the generator is broken down into modules. In the generator, we are going to preprocess the images as we have images of 2 different dimensions as well as create a batch of video frames. 

In [4]:
# Parameters initialization
nb_rows = 120   # X dimension of the image
nb_cols = 120   # Y dimesnion of the image

nb_frames = 30  # lenght of the video frames
nb_channel = 3 # numbe rof channels in images 3 for color(RGB) and 1 for Gray

In [5]:
# Helper function to generate a random affine transform on the iamge
def get_random_affine():
    dx, dy = np.random.randint(-1.7, 1.8, 2)
    M = np.float32([[1, 0, dx], [0, 1, dy]])
    return M

In [6]:
# # Helper function to normalise the data
# def normalize_data(data):
#     return data/127.5-1

In [7]:
# Helper function to initialize all the batch image data and labels
def initialize_batch_data(batch_size):
    batch_data = np.zeros((batch_size, nb_frames, nb_rows, nb_cols, nb_channel)) 
    batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
    return batch_data, batch_labels

In [8]:
def process_batch_images(source_path, folder_list, batch_num, batch_size, t,validation):
    
    batch_data,batch_labels = init_batch_data(batch_size)
    
    # Build an augumented batch data with affine transformation
    batch_data_aug,batch_labels_aug = initialize_batch_data(batch_size)
    
    # Build an augmented batch data with horizontal flip
    batch_data_flip,batch_labels_flip = initialize_batch_data(batch_size)
    
    #create a list of image numbers you want to use for a particular video using full frames
    img_idx = [x for x in range(0, nb_frames)] 

    for folder in range(batch_size): # iterate over the batch_size
        # read all the images in the folder
        imgs = sorted(os.listdir(source_path+'/'+ t[folder + (batch_num*batch_size)].split(';')[0])) 
        # Generate a random affine to be used in image transformation for buidling agumented data set
        M = get_random_affine()
        
        #  Iterate over the frames/images of a folder to read them in
        for idx, item in enumerate(img_idx): 
            ## image = imread(source_path+'/'+ t[folder + (batch_num*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
            image = cv2.imread(source_path+'/'+ t[folder + (batch_num*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # Cropping non symmetric frames
            if image.shape[0] != image.shape[1]:
                image=image[0:120,20:140]
            
            #crop the images and resize them. Note that the images are of 2 different shape 
            #and the conv3D will throw error if the inputs in a batch have different shapes   
            resized = cv2.resize(image, (nb_rows,nb_cols), interpolation = cv2.INTER_AREA)
            #Normal data
            batch_data[folder,idx] = (resized)
            
            #Data with affine transformation
            batch_data_aug[folder,idx] = (cv2.warpAffine(resized, M, (resized.shape[0], resized.shape[1])))
            
            # Data with horizontal flip
            batch_data_flip[folder,idx]= np.flip(resized,1)

        batch_labels[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
        batch_labels_aug[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
        
        # Labeling data with horizobtal flip, right swipe becomes left swipe and viceversa
        if int(t[folder + (batch_num*batch_size)].strip().split(';')[2])==0:
                    batch_labels_flip[folder, 1] = 1
        elif int(t[folder + (batch_num*batch_size)].strip().split(';')[2])==1:
                    batch_labels_flip[folder, 0] = 1
                    
        else:
                    batch_labels_flip[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
                  
    
    batch_data_final = np.append(batch_data, batch_data_aug, axis = 0)
    batch_data_final = np.append(batch_data_final, batch_data_flip, axis = 0)

    batch_labels_final = np.append(batch_labels, batch_labels_aug, axis = 0) 
    batch_labels_final = np.append(batch_labels_final, batch_labels_flip, axis = 0)
    
    if validation:
        batch_data_final=batch_data
        batch_labels_final= batch_labels
        
    return batch_data_final,batch_labels_final

In [9]:
def generator(source_path, folder_list, batch_size, validation=False):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            # you yield the batch_data and the batch_labels, remember what does yield do
            yield process_batch_images(source_path, folder_list, batch, batch_size, t,validation)          
        
        # Code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            batch_size = len(folder_list) - (batch_size*num_batches)
            yield process_batch_images(source_path, folder_list, batch, batch_size, t,validation)

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [10]:
curr_dt_time = datetime.datetime.now()
train_path = './Project_data/train'
val_path = './Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


## Model 4

In [11]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D, ZeroPadding3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam

#nb_filters = [8,16,32,64]
#nb_dense = [256, 128, 5]

nb_classes = 5

# Input
input_shape=(nb_frames,nb_rows,nb_cols,nb_channel)

# Define model
model = Sequential()
# 1st layer group
model.add(Conv3D(16, 3, 3, 3, activation='relu',
                         padding='same', name='conv1',
                         subsample=(1, 1, 1),
                         input_shape=input_shape))
model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2),
                               border_mode='valid', name='pool1'))
# 2nd layer group
model.add(Conv3D(32, 3, 3, 3, activation='relu',
                         padding='same', name='conv2',
                         subsample=(1, 1, 1)))
model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
                               border_mode='valid', name='pool2'))
# 3rd layer group
model.add(Conv3D(64, 3, 3, 3, activation='relu',
                         padding='same', name='conv3a',
                         subsample=(1, 1, 1)))
model.add(Conv3D(64, 3, 3, 3, activation='relu',
                         padding='same', name='conv3b',
                         subsample=(1, 1, 1)))
model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
                               border_mode='valid', name='pool3'))
# 4th layer group
model.add(Conv3D(128, 3, 3, 3, activation='relu',
                         padding='same', name='conv4a',
                         subsample=(1, 1, 1)))
model.add(Conv3D(128, 3, 3, 3, activation='relu',
                         padding='same', name='conv4b',
                         subsample=(1, 1, 1)))
model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
                               border_mode='valid', name='pool4'))

# 5th layer group
model.add(Conv3D(256, 3, 3, 3, activation='relu',
                         padding='same', name='conv5a',
                         subsample=(1, 1, 1)))
model.add(Conv3D(256, 3, 3, 3, activation='relu',
                         padding='same', name='conv5b',
                         subsample=(1, 1, 1)))
model.add(ZeroPadding3D(padding=(0, 1, 1)))
model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),
                               padding='valid', name='pool5'))
model.add(Flatten())

# FC layers group
model.add(Dense(512, activation='relu', name='fc6'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu', name='fc7'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes, activation='softmax'))

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(16, (3, 3, 3), activation="relu", padding="same", name="conv1", input_shape=(30, 120, ..., strides=(1, 1, 1))`
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: UserWarning: Update your `MaxPooling3D` call to the Keras 2 API: `MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name="pool1", padding="valid")`
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(32, (3, 3, 3), activation="relu", padding="same", name="conv2", strides=(1, 1, 1))`
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: UserWarning: Update your `MaxPooling3D` call to the Keras 2 API: `MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), name="pool2", padding="valid")`
/mnt/disks/user/anaconda3/lib/python3.6/site-pack

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [12]:
optimiser = Adam() #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 30, 120, 120, 16)  1312      
_________________________________________________________________
pool1 (MaxPooling3D)         (None, 30, 60, 60, 16)    0         
_________________________________________________________________
conv2 (Conv3D)               (None, 30, 60, 60, 32)    13856     
_________________________________________________________________
pool2 (MaxPooling3D)         (None, 15, 30, 30, 32)    0         
_________________________________________________________________
conv3a (Conv3D)              (None, 15, 30, 30, 64)    55360     
_________________________________________________________________
conv3b (Conv3D)              (None, 15, 30, 30, 64)    110656    
_________________________________________________________________
pool3 (MaxPooling3D)         (None, 7, 15, 15, 64)     0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [13]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size,validation=True)

In [14]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

# write the Reducelronplateau code here
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [15]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [17]:
batch_size = 10
num_epochs = 20
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  ./Project_data/val ; batch size = 10
Source path =  ./Project_data/train ; batch size = 10Epoch 1/20

67/67 [==============================] - 200s 3s/step - loss: 1.8782 - categorical_accuracy: 0.1944 - val_loss: 1.5621 - val_categorical_accuracy: 0.2800

Epoch 00001: saving model to model_init_2020-03-2908_17_13.927731/model-00001-1.88085-0.19407-1.56211-0.28000.h5
Epoch 2/20
67/67 [==============================] - 47s 707ms/step - loss: 1.5999 - categorical_accuracy: 0.2322 - val_loss: 1.6123 - val_categorical_accuracy: 0.2100

Epoch 00002: saving model to model_init_2020-03-2908_17_13.927731/model-00002-1.59992-0.23217-1.61225-0.21000.h5
Epoch 3/20
67/67 [==============================] - 47s 707ms/step - loss: 1.6140 - categorical_accuracy: 0.1692 - val_loss: 1.6103 - val_categorical_accuracy: 0.2200

Epoch 00003: saving model to model_init_2020-03-2908_17_13.927731/model-00003-1.61400-0.16915-1.61033-0.22000.h5

Epoch 00003: ReduceLROnPlateau reducing learning rat

In [18]:
batch_size = 20
num_epochs = 20

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20
34/34 [==============================] - 24s 707ms/step - loss: 1.6069 - categorical_accuracy: 0.2059 - val_loss: 1.6042 - val_categorical_accuracy: 0.2600

Epoch 00001: saving model to model_init_2020-03-2908_17_13.927731/model-00001-1.60692-0.20588-1.60421-0.26000.h5
Epoch 2/20
34/34 [==============================] - 24s 694ms/step - loss: 1.6087 - categorical_accuracy: 0.2059 - val_loss: 1.6167 - val_categorical_accuracy: 0.1400

Epoch 00002: saving model to model_init_2020-03-2908_17_13.927731/model-00002-1.60867-0.20588-1.61667-0.14000.h5
Epoch 3/20
34/34 [==============================] - 24s 695ms/step - loss: 1.6101 - categorical_accuracy: 0.2222 - val_loss: 1.6083 - val_categorical_accuracy: 0.2600

Epoch 00003: saving model to model_init_2020-03-2908_17_13.927731/model-00003-1.61008-0.22222-1.60827-0.26000.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 9.765625463842298e-07.
Epoch 4/20
34/34 [==============================] - 24s 696ms/step - loss: 1

In [19]:
batch_size = 30
num_epochs = 30

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
23/23 [==============================] - 17s 733ms/step - loss: 1.6024 - categorical_accuracy: 0.2850 - val_loss: 1.6076 - val_categorical_accuracy: 0.1750

Epoch 00001: saving model to model_init_2020-03-2908_17_13.927731/model-00001-1.60243-0.28502-1.60765-0.17500.h5
Epoch 2/30
23/23 [==============================] - 16s 705ms/step - loss: 1.6164 - categorical_accuracy: 0.0966 - val_loss: 1.6104 - val_categorical_accuracy: 0.2250

Epoch 00002: saving model to model_init_2020-03-2908_17_13.927731/model-00002-1.61638-0.09662-1.61043-0.22500.h5
Epoch 3/30
23/23 [==============================] - 16s 704ms/step - loss: 1.6012 - categorical_accuracy: 0.3140 - val_loss: 1.6094 - val_categorical_accuracy: 0.1500

Epoch 00003: saving model to model_init_2020-03-2908_17_13.927731/model-00003-1.60118-0.31401-1.60936-0.15000.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 3.814697446813398e-09.
Epoch 4/30
23/23 [==============================] - 16s 705ms/step - loss: 1

23/23 [==============================] - 16s 700ms/step - loss: 1.6129 - categorical_accuracy: 0.1691 - val_loss: 1.6067 - val_categorical_accuracy: 0.2500

Epoch 00027: saving model to model_init_2020-03-2908_17_13.927731/model-00027-1.61295-0.16908-1.60668-0.25000.h5

Epoch 00027: ReduceLROnPlateau reducing learning rate to 1.8626452377018543e-12.
Epoch 28/30
23/23 [==============================] - 16s 703ms/step - loss: 1.6102 - categorical_accuracy: 0.1884 - val_loss: 1.6075 - val_categorical_accuracy: 0.2500

Epoch 00028: saving model to model_init_2020-03-2908_17_13.927731/model-00028-1.61021-0.18841-1.60753-0.25000.h5
Epoch 29/30
23/23 [==============================] - 16s 705ms/step - loss: 1.6087 - categorical_accuracy: 0.1787 - val_loss: 1.6056 - val_categorical_accuracy: 0.2500

Epoch 00029: saving model to model_init_2020-03-2908_17_13.927731/model-00029-1.60869-0.17874-1.60558-0.25000.h5

Epoch 00029: ReduceLROnPlateau reducing learning rate to 9.313226188509272e-13.
Epo

In [20]:
batch_size = 50
num_epochs = 30

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
14/14 [==============================] - 10s 731ms/step - loss: 1.6050 - categorical_accuracy: 0.2381 - val_loss: 1.6159 - val_categorical_accuracy: 0.2000

Epoch 00001: saving model to model_init_2020-03-2908_17_13.927731/model-00001-1.60502-0.23810-1.61587-0.20000.h5
Epoch 2/30
14/14 [==============================] - 10s 697ms/step - loss: 1.6034 - categorical_accuracy: 0.2460 - val_loss: 1.6033 - val_categorical_accuracy: 0.2500

Epoch 00002: saving model to model_init_2020-03-2908_17_13.927731/model-00002-1.60339-0.24603-1.60330-0.25000.h5
Epoch 3/30
14/14 [==============================] - 10s 694ms/step - loss: 1.6084 - categorical_accuracy: 0.1905 - val_loss: 1.6107 - val_categorical_accuracy: 0.1500

Epoch 00003: saving model to model_init_2020-03-2908_17_13.927731/model-00003-1.60840-0.19048-1.61068-0.15000.h5
Epoch 4/30
14/14 [==============================] - 10s 693ms/step - loss: 1.6065 - categorical_accuracy: 0.2381 - val_loss: 1.6064 - val_categorical_accurac

14/14 [==============================] - 10s 696ms/step - loss: 1.6125 - categorical_accuracy: 0.1905 - val_loss: 1.6134 - val_categorical_accuracy: 0.1000

Epoch 00027: saving model to model_init_2020-03-2908_17_13.927731/model-00027-1.61246-0.19048-1.61341-0.10000.h5
Epoch 28/30
14/14 [==============================] - 10s 689ms/step - loss: 1.6118 - categorical_accuracy: 0.2143 - val_loss: 1.6068 - val_categorical_accuracy: 0.3000

Epoch 00028: saving model to model_init_2020-03-2908_17_13.927731/model-00028-1.61179-0.21429-1.60681-0.30000.h5

Epoch 00028: ReduceLROnPlateau reducing learning rate to 2.2737368624290214e-16.
Epoch 29/30
14/14 [==============================] - 10s 693ms/step - loss: 1.6067 - categorical_accuracy: 0.2381 - val_loss: 1.6092 - val_categorical_accuracy: 0.2000

Epoch 00029: saving model to model_init_2020-03-2908_17_13.927731/model-00029-1.60669-0.23810-1.60918-0.20000.h5
Epoch 30/30
14/14 [==============================] - 10s 695ms/step - loss: 1.6066 -

In [21]:
batch_size = 90
num_epochs = 30

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30
8/8 [==============================] - 6s 789ms/step - loss: 1.6123 - categorical_accuracy: 0.1944 - val_loss: 1.6020 - val_categorical_accuracy: 0.3500

Epoch 00001: saving model to model_init_2020-03-2908_17_13.927731/model-00001-1.61228-0.19444-1.60202-0.35000.h5
Epoch 2/30
8/8 [==============================] - 6s 729ms/step - loss: 1.6034 - categorical_accuracy: 0.2361 - val_loss: 1.6122 - val_categorical_accuracy: 0.1000

Epoch 00002: saving model to model_init_2020-03-2908_17_13.927731/model-00002-1.60339-0.23611-1.61223-0.10000.h5
Epoch 3/30
8/8 [==============================] - 6s 727ms/step - loss: 1.6121 - categorical_accuracy: 0.0694 - val_loss: 1.6054 - val_categorical_accuracy: 0.2000

Epoch 00003: saving model to model_init_2020-03-2908_17_13.927731/model-00003-1.61210-0.06944-1.60538-0.20000.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 5.684342156072553e-17.
Epoch 4/30
8/8 [==============================] - 6s 729ms/step - loss: 1.6061 - cate

8/8 [==============================] - 6s 730ms/step - loss: 1.6008 - categorical_accuracy: 0.2222 - val_loss: 1.6111 - val_categorical_accuracy: 0.2500

Epoch 00027: saving model to model_init_2020-03-2908_17_13.927731/model-00027-1.60075-0.22222-1.61111-0.25000.h5

Epoch 00027: ReduceLROnPlateau reducing learning rate to 1.3877788466974007e-20.
Epoch 28/30
8/8 [==============================] - 6s 731ms/step - loss: 1.6061 - categorical_accuracy: 0.2222 - val_loss: 1.6046 - val_categorical_accuracy: 0.3000

Epoch 00028: saving model to model_init_2020-03-2908_17_13.927731/model-00028-1.60610-0.22222-1.60463-0.30000.h5
Epoch 29/30
8/8 [==============================] - 6s 730ms/step - loss: 1.6027 - categorical_accuracy: 0.2917 - val_loss: 1.6117 - val_categorical_accuracy: 0.1500

Epoch 00029: saving model to model_init_2020-03-2908_17_13.927731/model-00029-1.60270-0.29167-1.61168-0.15000.h5

Epoch 00029: ReduceLROnPlateau reducing learning rate to 6.938894233487004e-21.
Epoch 30/30


In [23]:
batch_size = 90
num_epochs = 20

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    

model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20
8/8 [==============================] - 7s 834ms/step - loss: 1.6002 - categorical_accuracy: 0.2778 - val_loss: 1.6039 - val_categorical_accuracy: 0.3000

Epoch 00001: saving model to model_init_2020-03-2908_17_13.927731/model-00001-1.60023-0.27778-1.60394-0.30000.h5
Epoch 2/20
8/8 [==============================] - 6s 741ms/step - loss: 1.6065 - categorical_accuracy: 0.2639 - val_loss: 1.6131 - val_categorical_accuracy: 0.1500

Epoch 00002: saving model to model_init_2020-03-2908_17_13.927731/model-00002-1.60655-0.26389-1.61305-0.15000.h5
Epoch 3/20
8/8 [==============================] - 6s 746ms/step - loss: 1.6025 - categorical_accuracy: 0.3194 - val_loss: 1.6090 - val_categorical_accuracy: 0.2000

Epoch 00003: saving model to model_init_2020-03-2908_17_13.927731/model-00003-1.60249-0.31944-1.60902-0.20000.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 1.0842022239823443e-22.
Epoch 4/20
8/8 [==============================] - 6s 738ms/step - loss: 1.6043 - cat